In [1]:
# import schedule
# import time
# import os

# def data_generator():
#     print("run")

# schedule.every(10).seconds.do(data_generator)

# while 1:
#     schedule.run_pending()
#     time.sleep(1)

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.context import SparkContext

from LockQueue_module import LockQueue
from SparkClusterJob import SparkClusterJob

import string
from pymongo import MongoClient
import pandas as pd
import schedule
import time
import os

# import json
# from bson import ObjectId

# class JSONEncoder(json.JSONEncoder):
#     def default(self, o):
#         if isinstance(o, ObjectId):
#             return str(o)
#         return json.JSONEncoder.default(self, o)

class DataHolder:

    data = []



def cleaning_func(x):

    # lower case
    # replace & with and
    # remove punctuation
    
    x["title"] = x["title"].lower().replace("&","and").translate(str.maketrans('', '', string.punctuation))
    if str(x["current_price"]).lower() == 'nan':
        x["current_price"] = 0

    if str(x["previous_price"]).lower() == 'nan':
        x["previous_price"] = 0

    return x
    # return JSONEncoder().encode(x["fullDocument"])

def execute_spark_job():
    print("------------------- new job")
    lq = LockQueue()
    
    dh = DataHolder()
    
    while True:
        dh.data = lq.get_read_queue()
    
        if(type(dh.data) == type([])):
            break
            
    if(dh.data !=[]):
    
        scj = SparkClusterJob()
    
        while True:
            print("Waiting for previous job to complete")
            if scj.set_job_status():
                print("Got use Loc for Spark Cluster")
                break
    
        
        print("Length of the data : ",len(dh.data))
    
        
        spark = SparkSession \
        .builder \
        .appName("mongodbtest1") \
        .master("spark://10.1.174.144:7077")\
        .getOrCreate()
        # .config("spark.mongodb.read.uri", "mongodb://abhi_1:60000/shardDB.asofashion_2") \
        # .config("spark.mongodb.write.uri", "mongodb://abhi_1:60000/shardDB.asofashion_clean") \
        # .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.13:10.2.2') \
        # .getOrCreate()
    
        clean_doc = spark.sparkContext.parallelize(dh.data).map(cleaning_func).collect()
    
        client = MongoClient("149.165.172.75", 60000)
        db = client.shardDB
        
        collection = db["asofashion_clean"]
    
    
        collection.insert_many(clean_doc)
        spark.stop()
        scj.release_job_status()
    
        
    else:
        pass
        print("No Data for Spark Job to Execute")

    print("------------------- job complete")

schedule.every(37).seconds.do(execute_spark_job)

while 1:
    schedule.run_pending()
    time.sleep(1)






------------------- new job
No Data for Spark Job to Execute
------------------- job complete
------------------- new job
No Data for Spark Job to Execute
------------------- job complete
------------------- new job
Waiting for previous job to complete
Got use Loc for Spark Cluster
Length of the data :  134


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/19 18:11:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


------------------- job complete
------------------- new job
Waiting for previous job to complete
Got use Loc for Spark Cluster
Length of the data :  1365


------------------- job complete
------------------- new job
Waiting for previous job to complete
Got use Loc for Spark Cluster
Length of the data :  998


------------------- job complete
------------------- new job
Waiting for previous job to complete
Got use Loc for Spark Cluster
Length of the data :  1003


...

------------------- job complete
------------------- new job
Waiting for previous job to complete
Got use Loc for Spark Cluster
Length of the data :  1004


------------------- job complete
------------------- new job
Waiting for previous job to complete
Got use Loc for Spark Cluster
Length of the data :  998


------------------- job complete
------------------- new job
Waiting for previous job to complete
Got use Loc for Spark Cluster
Length of the data :  4


------------------- job complete
------------------- new job
No Data for Spark Job to Execute
------------------- job complete


AttributeError: 'NoneType' object has no attribute 'sc'

In [8]:
import pickle

with open("queue.pkl","wb") as f:
    pickle.dump([],f)

In [9]:
import pickle

with open("queue.pkl","rb") as f:
    r = pickle.load(f)
    print(r)

[]


In [ ]:
# import gzip
# import ubjson


# for _ in range(1,10000):

#     with gzip.open("/home/exouser/spark_pyjobs/queue.pkl", 'rb') as f:
#         l = ubjson.load(f)
#         # print(len(l))
    
#     with gzip.open("/home/exouser/spark_pyjobs/queue.pkl", 'wb') as f:
#         ubjson.dump(l+[{'_id': "ObjectId('66199e713b0b0281eac0d129')", 'product_id': 202091136, 'brand_name': 'adidas Originals', 'title': 'adidas Originals Adventure mountain t-shirt in black', 'current_price': 31.58, 'previous_price': 52.11, 'colour': "nan", 'currency': 'USD', 'rrp': "nan", 'productCode': 115601763, 'productType': 'Product', 'gender': 'male'} ]
#     , f)

In [ ]:
# with gzip.open("/home/exouser/spark_pyjobs/queue.pkl", 'rb') as f:
#     l = ubjson.load(f)
#     print(len(l))